In [1]:
import uuid
from datetime import datetime

def get_data():
    import requests

    res = requests.get("https://randomuser.me/api/")
    res = res.json()
    res = res['results'][0]

    return res
    # print(json.dumps(res, indent=3))

def format_data(res):
    data = {}
    location = res['location']
    data['id'] = uuid.uuid4().int
    data['first_name'] = res['name']['first']
    data['last_name'] = res['name']['last']
    data['gender'] = res['gender']
    data['address'] = f"{str(location['street']['number'])} {location['street']['name']}, " \
                      f"{location['city']}, {location['state']}, {location['country']}"
    data['post_code'] = location['postcode']
    data['email'] = res['email']
    data['username'] = res['login']['username']
    data['dob'] = res['dob']['date']
    data['registered_date'] = res['registered']['date']
    data['phone'] = res['phone']
    data['picture'] = res['picture']['medium']
    return data


In [2]:
import json
from kafka import KafkaProducer
import time
import logging

# producer = KafkaProducer(bootstrap_servers=['broker:29092'], max_block_ms=5000)
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], max_block_ms=5000)
curr_time = time.time()

try:
    res = get_data()
    res = format_data(res)
    print(res)

    producer.send('users_created', json.dumps(res).encode('utf-8'))
except Exception as e:
        logging.error(f'An error occured: {e}')

{'id': 247817924588245923341384306894570166991, 'first_name': 'Ida', 'last_name': 'Nguyen', 'gender': 'female', 'address': '8085 Pockrus Page Rd, Roseville, California, United States', 'post_code': 36146, 'email': 'ida.nguyen@example.com', 'username': 'redlion718', 'dob': '1982-08-07T00:02:02.743Z', 'registered_date': '2009-05-16T23:40:21.802Z', 'phone': '(477) 278-5931', 'picture': 'https://randomuser.me/api/portraits/med/women/10.jpg'}
